<a href="https://colab.research.google.com/github/shin0105/4YP/blob/master/RFregression_Boston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bayesian-optimization
!pip install GPyOpt

In [12]:
#import packages
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn import ensemble
from sklearn.metrics import mean_squared_error,accuracy_score
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

In [3]:
#reading csv file for Boston Housing Dataset
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['hou_all.csv']),\
                 names = ['CRIM','ZN','INDUS','CHAS','NOS','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV'])
df.head()

Saving hou_all.csv to hou_all.csv


,CRIM,ZN,INDUS,CHAS,NOS,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
#parameters = CRIM , ZN, INDUS
X = df.iloc[:,0:3].to_numpy()

#target = MEDV
Y = df.iloc[:,-1].to_numpy()

#train test split
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.3, random_state = 10)

#building rf regression model
rf = RandomForestRegressor()
rf.fit(xTrain,yTrain)
yPred = rf.predict(xTest)

#calculating loss - MSE
MSE = mean_squared_error(yPred,yTest)

In [10]:
def bo_params_rf(max_samples,n_estimators,max_features):
    
    params = {
        'max_samples': max_samples,
        'max_features':max_features,
        'n_estimators':int(n_estimators)
    }
    clf = RandomForestRegressor(max_samples=params['max_samples'],max_features=params['max_features'],n_estimators=params['n_estimators'])
    clf.fit(xTrain,yTrain)
    yPred = clf.predict(xTest)
    
    #using negative MSE as score (loss function)
    score = -mean_squared_error(yPred,yTest)
    return score

In [33]:
#boundary for Bayesian optimization
bounds = {'max_samples':(0.5,1),
          'max_features':(0.5,1),
          'n_estimators':(100,200)}

rf_bo = BayesianOptimization(bo_params_rf, bounds)

results = rf_bo.maximize(n_iter=20, init_points=5)

|   iter    |  target   | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------
| 1         | -62.73    | 0.9463    | 0.7309    | 139.8     |
| 2         | -62.38    | 0.8209    | 0.6473    | 186.6     |
| 3         | -64.12    | 0.8604    | 0.9446    | 163.9     |
| 4         | -60.9     | 0.5147    | 0.5962    | 129.4     |
| 5         | -62.37    | 0.5308    | 0.795     | 127.9     |
| 6         | -65.68    | 0.9592    | 0.9663    | 130.0     |
| 7         | -63.52    | 0.647     | 0.8853    | 132.7     |
| 8         | -63.41    | 0.731     | 0.7409    | 155.6     |
| 9         | -64.01    | 0.5582    | 0.936     | 135.2     |
| 10        | -63.52    | 0.7318    | 0.7543    | 195.4     |
| 11        | -58.75    | 0.7204    | 0.5093    | 114.2     |
| 12        | -59.74    | 0.5955    | 0.5165    | 154.8     |
| 13        | -61.77    | 0.9373    | 0.5694    | 131.1     |
| 14        | -62.59    | 0.9786    | 0.6613    | 171.8     |
| 15    

In [34]:
params = rf_bo.max['params']
params['n_estimators']= int(params['n_estimators'])
print(params)

{'max_features': 0.7203624219872533, 'max_samples': 0.5093120201090363, 'n_estimators': 114}


In [40]:
#using GPyOpt
import GPyOpt

bounds = [ {'name': 'max_samples', 'type': 'continuous', 'domain': (0.5, 1)},
        {'name': 'max_features', 'type': 'continuous', 'domain': (0.5, 1)},
        {'name': 'n_estimators', 'type': 'discrete', 'domain': (100, 200)}
      ]

def rf_score(parameters):
  parameters = parameters[0]
  params = {
        'max_samples': parameters[0],
        'max_features':parameters[1],
        'n_estimators':int(parameters[2])
    }
  
  clf = RandomForestRegressor(max_samples=params['max_samples'],max_features=params['max_features'],n_estimators=params['n_estimators'])
  clf.fit(xTrain,yTrain)
  yPred = clf.predict(xTest)
    
  #using negative MSE as score (loss function)
  score = -mean_squared_error(yPred,yTest)
  return score

In [41]:
optimizer = GPyOpt.methods.BayesianOptimization(f=rf_score, 
                                                   domain=bounds,
                                                   acquisition_type ='EI',
                                                   initial_design_numdata = 5,
                                                   model_type='GP',
                                                   acquisition_jitter = 0.05,
                                                   maximize=True
                                                   )

optimizer.run_optimization(max_iter=20)

#minimum MSE
MSE_min=-optimizer.fx_opt
#best parameters
params_min=optimizer.x_opt
print(MSE_min)
print(params_min)

-59.72452424342105
[  0.61149139   0.85901991 100.        ]
